In [1]:
import codecs
from collections import Counter
import string
from Crypto.Cipher import AES
from Crypto import Random
import random as rnd
import time
import hashlib
import requests

# those variables are used later in several problems so I just initialize them here.
secret_key = Random.get_random_bytes(16)
iv_secret = Random.get_random_bytes(16)
secret_key_16_bits = Random.get_random_bytes(2)

Copied from cryptopals.com
## Crypto Challenge Set 1

This is the qualifying set. We picked the exercises in it to ramp developers up gradually into coding cryptography, but also to verify that we were working with people who were ready to write code.

This set is relatively easy. With one exception, most of these exercises should take only a couple minutes. But don't beat yourself up if it takes longer than that. It took Alex two weeks to get through the set!

If you've written any crypto code in the past, you're going to feel like skipping a lot of this. Don't skip them. At least two of them (we won't say which) are important stepping stones to later attacks. 


Convert hex to base64

The string:
```
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
```
Should produce:
```
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
```
So go ahead and make that happen. You'll need to use this code for the rest of the exercises.

Cryptopals Rule
```
Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.
```

In [2]:
## Challenge 1 convert from hex to base64
## This one is pretty straightforward
input = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
raw = codecs.decode(input, "hex")
result = codecs.encode(raw,"base64")
expected_result = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
result.strip().decode('ascii') == expected_result

True